## Import data

In [17]:
!pip uninstall -y numpy pandas_ta
!pip install numpy==1.23.5 pandas-ta

!pip install backtesting
!pip install pandas_ta
!pip install hyperopt

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas_ta 0.3.14b0
Uninstalling pandas_ta-0.3.14b0:
  Successfully uninstalled pandas_ta-0.3.14b0
  Using cached pandas_ta-0.3.14b0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 77.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires 

In [1]:
!pip install pandas_ta

In [2]:
import pandas as pd

df1 = pd.read_csv('tradingview_signals_server.csv')
df2 = pd.read_csv('tradingview_signals.csv')
df3 = pd.read_csv('tradingview_signals_first.csv')

df = pd.concat([df1, df2, df3], ignore_index=True)

df = df.sort_values(by='utc_time')
df.to_csv('tradingview_signals_merged.csv', index=False)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 56872 entries, 56257 to 55418
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   #               56872 non-null  int64  
 1   id              56872 non-null  int64  
 2   symbol          56872 non-null  object 
 3   timeframe       56872 non-null  object 
 4   signal          56872 non-null  object 
 5   entry_price     56872 non-null  float64
 6   unix_timestamp  56872 non-null  int64  
 7   utc_time        56872 non-null  object 
 8   local_time      56872 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 4.3+ MB


,#,id,symbol,timeframe,signal,entry_price,unix_timestamp,utc_time,local_time
56257,1,27,MASKUSDT,1h,STRONG_BUY,2.263938,1740502902,2025-02-26 00:01:42.899724,2025-02-26 00:01:42.899724
56258,2,28,ETHUSDT,1h,SELL,2420.460000,1740502930,2025-02-26 00:02:10.098242,2025-02-26 00:02:10.098242
56259,3,29,COWUSDT,1h,STRONG_BUY,0.444927,1740502967,2025-02-26 00:02:47.768526,2025-02-26 00:02:47.768526
56260,4,30,BTCUSDT,1h,SELL,87557.663770,1740502983,2025-02-26 00:03:03.139575,2025-02-26 00:03:03.139575
56261,5,31,PENGUUSDT,15m,STRONG_BUY,0.008166,1740504607,2025-02-26 00:30:07.400803,2025-02-26 00:30:07.400803
...,...,...,...,...,...,...,...,...,...
55413,55414,55414,ETCUSDT,30m,STRONG_SELL,17.350639,1742311841,2025-03-18 15:30:41.000000,2025-03-18 22:30:41.000000
55417,55418,55418,SSVUSDT,30m,STRONG_SELL,7.437548,1742311842,2025-03-18 15:30:42.000000,2025-03-18 22:30:42.000000
55419,55420,55420,CHZUSDT,30m,STRONG_SELL,0.044696,1742311842,2025-03-18 15:30:42.000000,2025-03-18 22:30:42.000000
55416,55417,55417,USTCUSDT,30m,STRONG_SELL,0.012629,1742311842,2025-03-18 15:30:42.000000,2025-03-18 22:30:42.000000


In [13]:
import pandas as pd
import numpy as np

interval = "30m"
klines_df = pd.read_csv(f"klines_data_{interval}.csv")
# signals_df = pd.read_csv("tradingview_signals_server.csv")
signals_df = df.copy()

signals_df["utc_time"] = pd.to_datetime(signals_df["unix_timestamp"], unit="s", utc=True).dt.floor("H")

print(signals_df[["symbol", "utc_time", "signal"]].head())


print('klines_df.shape', klines_df.shape)
klines_df.info()

          symbol                  utc_time      signal
56257   MASKUSDT 2025-02-25 17:00:00+00:00  STRONG_BUY
56258    ETHUSDT 2025-02-25 17:00:00+00:00        SELL
56259    COWUSDT 2025-02-25 17:00:00+00:00  STRONG_BUY
56260    BTCUSDT 2025-02-25 17:00:00+00:00        SELL
56261  PENGUUSDT 2025-02-25 17:00:00+00:00  STRONG_BUY
klines_df.shape (315534, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315534 entries, 0 to 315533
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    315534 non-null  object 
 1   Open    315534 non-null  float64
 2   High    315534 non-null  float64
 3   Low     315534 non-null  float64
 4   Close   315534 non-null  float64
 5   Volume  315534 non-null  float64
 6   Symbol  315534 non-null  object 
dtypes: float64(5), object(2)
memory usage: 16.9+ MB


<ipython-input-13-000958de1976>:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  signals_df["utc_time"] = pd.to_datetime(signals_df["unix_timestamp"], unit="s", utc=True).dt.floor("H")


In [14]:
print('signals_df.shape', signals_df.shape)
signals_df.info()

signals_df.shape (56872, 9)
<class 'pandas.core.frame.DataFrame'>
Index: 56872 entries, 56257 to 55418
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   #               56872 non-null  int64              
 1   id              56872 non-null  int64              
 2   symbol          56872 non-null  object             
 3   timeframe       56872 non-null  object             
 4   signal          56872 non-null  object             
 5   entry_price     56872 non-null  float64            
 6   unix_timestamp  56872 non-null  int64              
 7   utc_time        56872 non-null  datetime64[ns, UTC]
 8   local_time      56872 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(3), object(4)
memory usage: 6.4+ MB


In [15]:
klines_df['Date'] = pd.to_datetime(klines_df['Date'])
signals_df['utc_time'] = pd.to_datetime(signals_df['utc_time'])

signals_df = signals_df[signals_df["timeframe"] == f"{interval}"]

## Hyperopt_by_month

In [ ]:
# !pip install backtesting
# !pip install pandas_ta
# !pip install hyperopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.1/190.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=3aa5a34d2dadafebc02a9f69d074c98fdfbc644b3bceed317fd68190166c4e4d
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [ ]:
import pandas as pd
from datetime import datetime
from hyperopt import fmin, tpe, hp, Trials
import backtesting
from dateutil.relativedelta import relativedelta
import traceback

def generate_monthly_intervals(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m")
    end = datetime.strptime(end_date, "%Y-%m")
    intervals = []
    while start <= end:
        next_month = start + relativedelta(months=1)
        intervals.append((start.strftime("%Y-%m"), next_month.strftime("%Y-%m")))
        start = next_month
    return intervals

def full_period_backtest(strategy, data, initial_capital=1_000_000):
    data.index = pd.to_datetime(data.index).tz_localize(None)
    test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=0.0005)
    res = test.run()
    trades = res._trades.copy()
    # print(res)
    return trades, res

from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd

def analyze_monthly_results(trades, intervals, initial_capital, equity_curve):
    trades['ExitTime'] = pd.to_datetime(trades['ExitTime']).dt.tz_localize(None)
    trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
    trades.set_index('Close Time', inplace=True)
    trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

    monthly_results = {}
    total_pnl = 0
    profit_months = 0
    loss_months = 0
    total_trade_duration = 0
    total_trades = len(trades)

    capital = initial_capital  # Начальный капитал
    last_interval = intervals[-1]  # Последний месяц

    for start, end in intervals:
        start_date = datetime.strptime(start, "%Y-%m")
        end_date = datetime.strptime(end, "%Y-%m") + relativedelta(months=1, days=-1, hours=23, minutes=59, seconds=59)

        monthly_trades = trades[(trades.index >= start_date) & (trades.index <= end_date)]

        if monthly_trades.empty:
            monthly_results[start] = (0.0, 0)
            continue

        # PnL за месяц
        monthly_pnl = monthly_trades['PnL'].sum()
        capital += monthly_pnl  # Обновляем капитал только на PnL

        # Если это последний месяц, учитываем открытые позиции
        if (start, end) == last_interval and not equity_curve.empty:
            last_equity = equity_curve['Equity'].iloc[-1]  # Финальная Equity
            open_pnl = last_equity - capital  # Открытые позиции
            capital += open_pnl  # Учитываем их в капитале

        # Итоговый процент профита за месяц
        prev_capital = capital - monthly_pnl  # Капитал до учета текущего месяца
        monthly_pnl_percent = ((capital - prev_capital) / prev_capital) * 100 if prev_capital != 0 else 0

        # Средняя длительность сделки (в часах)
        avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600 if len(monthly_trades) > 0 else 0

        monthly_results[start] = (round(monthly_pnl_percent, 8), round(avg_trade_duration, 2))

        total_pnl += monthly_pnl
        total_trade_duration += monthly_trades['Trade Duration'].sum()

        if monthly_pnl_percent > 0:
            profit_months += 1
        else:
            loss_months += 1

    avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0
    total_months = profit_months + loss_months
    profit_month_ratio = (profit_months / total_months * 100) if total_months > 0 else 0

    # Итоговый Return [%]
    total_return = ((capital - initial_capital) / initial_capital) * 100

    return total_return, profit_months, loss_months, monthly_results, avg_trade_duration_total, profit_month_ratio



def generate_search_space(params_to_optimize):
    space = {}
    for param, settings in params_to_optimize.items():
        if settings['type'] == 'quniform':
            space[param] = hp.quniform(param, settings['range'][0], settings['range'][1], settings.get('q', 1))
        elif settings['type'] == 'uniform':
            space[param] = hp.uniform(param, settings['range'][0], settings['range'][1])
        else:
            raise ValueError(f"Неизвестный тип параметра: {settings['type']}")
    return space

def optimize_strategy(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals, params_to_optimize, fixed_params, strategy):
    intervals = generate_monthly_intervals(start_date, end_date)
    trials = Trials()
    space = generate_search_space(params_to_optimize)
    results = []

    print(f"Общее количество строк в загруженных данных: {len(data)}")
    print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
    print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

    def objective(params):
        for key, value in fixed_params.items():
            setattr(strategy, key, value)

        for key, value in params.items():
            setattr(strategy, key, int(value) if 'length' in key else value)

        print(f"Параметры: {params}, {fixed_params}")

        try:
            trades, res = full_period_backtest(strategy, data, 1_000_000)

            total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration, profit_month_ratio = analyze_monthly_results(
                trades, intervals, 1_000_000, res._equity_curve
            )

            print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}, Процент профитных месяцев: {profit_month_ratio:.2f}%")
            print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

            row = {
                'symbol': symbol,
                'interval': interval,
                **{key: fixed_params.get(key, None) for key in fixed_params},
                **{key: (int(value) if 'length' in key else value) for key, value in params.items()},
                'strategy': strategy.__name__,
                'total_pnl%': total_pnl,
                'avg_trade_duration_hours': avg_trade_duration,
                'profit_month_count': profit_months,
                'loss_month_count': loss_months,
                'profit_month_ratio%': profit_month_ratio
            }

            for month, (pnl, _) in monthly_results.items():
                row[month] = pnl

            results.append(row)
            return {'loss': -total_pnl, 'status': 'ok', 'params': params}

        except Exception as e:
            print("Ошибка в objective:", e)
            print(traceback.format_exc())  # Логирование полной трассировки ошибки
            return {'loss': float('inf'), 'status': 'fail', 'params': params}

    try:
        fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    except Exception as e:
        print("Ошибка при выполнении fmin:", e)
        print(traceback.format_exc())
        return None

    results_df = pd.DataFrame(results)
    save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

    if trials.trials:
        best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
        best_params = best_trial['result']['params']
        best_loss = best_trial['result']['loss']
        return best_params, -best_loss
    else:
        print("Не удалось найти лучшие параметры.")
        return None


# best_params, total_pnl = optimize_strategy(
#     data=df,
#     symbol=symbol,
#     interval=interval,
#     start_date=start_date,
#     end_date=end_date,
#     max_evals=max_evals,
#     shared_folder_path=shared_folder_path,
#     params_to_optimize=params_to_optimize,
#     fixed_params=fixed_params
# )

# # Настройка стратегии с оптимизированными параметрами
# SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
# SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
# SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
# SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
# SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

# print("\nНастройка стратегии завершена.")

# # Тестируем стратегию
# test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
# res = test.run()

# print(symbol)
# print()
# print("Результаты с оптимизированными параметрами:", res)
# print('Window length', SlopingStrategy.window_length)
# print('Min space', SlopingStrategy.min_space)
# print('Sloping atr length', SlopingStrategy.sloping_atr_length)
# print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
# print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


## Strategy

In [7]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy

trade_log = []
class SignalStrategy(Strategy):
    stop_loss_multiplier = 0.4
    take_profit_multiplier = 1.2
    signals_df = None
    symbol = None

    def init(self):
        """Load trading signals and ATR indicator"""
        self.signals = self.signals_df[self.signals_df["symbol"] == self.symbol]
        self.used_signals = set()

        def atr_indicator(h, l, c):
            """Calculate ATR (Average True Range)"""
            try:
                atr_values = ta.atr(pd.Series(h), pd.Series(l), pd.Series(c), length=14)
                return atr_values.fillna(0).to_numpy() if atr_values is not None else np.zeros(len(h))
            except Exception as e:
                print(f"Error calculating ATR: {e}")
                return np.zeros(len(h))

        self.atr = self.I(atr_indicator, self.data.High, self.data.Low, self.data.Close)

    def next(self):
        """Process signals and open trades accordingly"""
        current_time = self.data.index[-1]
        current_price = self.data.Close[-1]
        current_atr = self.atr[-1]

        if current_atr == 0 or pd.isna(current_atr):
            return

        for _, signal in self.signals.iterrows():
            signal_time = signal['utc_time']

            if signal_time in self.used_signals:
                continue

            if signal_time in self.data.index:
                self.used_signals.add(signal_time)

                if signal['signal'] == "STRONG_BUY" and not self.position:
                    stop_loss = current_price - (self.stop_loss_multiplier * current_atr)
                    take_profit = current_price + (self.take_profit_multiplier * current_atr)

                    if stop_loss < current_price < take_profit:
                        print(f"🚀 LONG {self.symbol} | {current_time} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
                        self.buy(sl=stop_loss, tp=take_profit)

                elif signal['signal'] == "STRONG_SELL" and not self.position:
                    stop_loss = current_price + (self.stop_loss_multiplier * current_atr)
                    take_profit = current_price - (self.take_profit_multiplier * current_atr)

                    if take_profit < current_price < stop_loss:
                        print(f"🚨 SHORT {self.symbol} | {current_time} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
                        self.sell(sl=stop_loss, tp=take_profit)

    def notify_trade(self, trade):
        """Log trade results when a trade is closed"""
        if trade.is_closed:
            trade_log.append({
                "Symbol": self.symbol,
                "Entry Time": trade.entry_time,
                "Exit Time": trade.exit_time,
                "Entry Price": trade.entry_price,
                "Exit Price": trade.exit_price,
                "Return [%]": trade.pl_pct * 100,
            })


/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


### Backtest tradingview

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy


trade_log = []

class SignalStrategy(Strategy):
    stop_loss_multiplier = 0.4
    take_profit_multiplier = 1.2
    signals_df = None
    btc_signals_df = None
    symbol = None

    def init(self):
        """Load trading signals and ATR indicator"""
        self.signals = self.signals_df[self.signals_df["symbol"] == self.symbol].copy()
        self.used_signals = set()

        # Обрабатываем BTC сигналы
        self.btc_signals = self.signals_df[self.signals_df["symbol"] == "BTCUSDT"].copy()
        if not self.btc_signals.empty:
            self.btc_signals["utc_time"] = pd.to_datetime(self.btc_signals["utc_time"]).dt.floor("min")
        else:
            self.btc_signals = None  # Отключаем проверку, если данных нет
            # print("⚠️ Нет сигналов BTCUSDT! Проверка отключена.")

        self.signals["utc_time"] = pd.to_datetime(self.signals["utc_time"]).dt.floor("min")

        # ATR Indicator
        self.atr = self.I(self.atr_indicator, self.data.High, self.data.Low, self.data.Close)

    def atr_indicator(self, h, l, c):
        """Calculate ATR (Average True Range)"""
        try:
            atr_values = ta.atr(pd.Series(h), pd.Series(l), pd.Series(c), length=14)
            return atr_values.fillna(0).to_numpy() if atr_values is not None else np.zeros(len(h))
        except Exception as e:
            print(f"Error calculating ATR: {e}")
            return np.zeros(len(h))

    # def next(self):
    #     """Process signals and open trades accordingly"""
    #     current_time = self.data.index[-1]
    #     current_price = self.data.Close[-1]
    #     current_atr = self.atr[-1]

    #     if current_atr == 0 or pd.isna(current_atr):
    #         return

    #     # Получаем сигнал BTC на текущий момент
    #     btc_signal = None
    #     if self.btc_signals is not None:
    #         btc_row = self.btc_signals[self.btc_signals["utc_time"] == current_time]
    #         if not btc_row.empty:
    #             btc_signal = btc_row.iloc[0]["signal"]

    #     print(f"[{current_time}] 🔍 BTC сигнал: {btc_signal if btc_signal else 'Нет сигнала'}")

    #     # Фильтруем сигналы текущего символа по времени
    #     relevant_signals = self.signals[self.signals["utc_time"] == current_time]

    #     for _, signal in relevant_signals.iterrows():
    #         if signal["utc_time"] in self.used_signals:
    #             continue

    #         self.used_signals.add(signal["utc_time"])

    #         if btc_signal is None:
    #             print(f"[{current_time}] ❌ Нет BTC сигнала, пропускаем {self.symbol}")
    #             continue

    #         if (btc_signal in ["BUY", "STRONG_BUY", "NEUTRAL"] and signal["signal"] == "STRONG_BUY"):
    #             stop_loss = current_price - (self.stop_loss_multiplier * current_atr)
    #             take_profit = current_price + (self.take_profit_multiplier * current_atr)
    #             print(f"🚀 LONG {self.symbol} | {current_time} | BTC: {btc_signal} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
    #             self.buy(sl=stop_loss, tp=take_profit)

    #         elif (btc_signal in ["SELL", "STRONG_SELL", "NEUTRAL"] and signal["signal"] == "STRONG_SELL"):
    #             stop_loss = current_price + (self.stop_loss_multiplier * current_atr)
    #             take_profit = current_price - (self.take_profit_multiplier * current_atr)
    #             print(f"🚨 SHORT {self.symbol} | {current_time} | BTC: {btc_signal} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
    #             self.sell(sl=stop_loss, tp=take_profit)


    def next(self):
        """Обрабатываем сигналы, если они соответствуют таймфрейму"""
        current_time = self.data.index[-1]  # Время текущей свечи
        current_price = self.data.Close[-1]
        current_atr = self.atr[-1]

        if current_atr == 0 or pd.isna(current_atr):
            return



        # Получаем BTC сигнал, если он есть
        btc_signal = None
        if self.btc_signals is not None:
            btc_row = self.btc_signals[(self.btc_signals["utc_time"] == current_time) &
                                      (self.btc_signals["timeframe"] == interval)]
            if not btc_row.empty:
                btc_signal = btc_row.iloc[0]["signal"]

        # print(f"[{current_time}] 🔍 BTC сигнал: {btc_signal if btc_signal else 'Нет сигнала'}")

        # Фильтруем сигналы по `utc_time` и `timeframe == interval`
        relevant_signals = self.signals[(self.signals["utc_time"] == current_time) &
                                        (self.signals["timeframe"] == interval)]

        if relevant_signals.empty:
            # print(f"[{current_time}] ⏳ Нет сигналов {self.symbol} на интервале {interval}")
            return  # Пропускаем, если нет сигналов в нужном интервале

        for _, signal in relevant_signals.iterrows():
            if signal["utc_time"] in self.used_signals:
                continue

            self.used_signals.add(signal["utc_time"])

            # Убеждаемся, что BTC сигнал тоже соответствует интервалу
            if btc_signal is None:
                # print(f"[{current_time}] ❌ Нет BTC сигнала, пропускаем {self.symbol}")
                continue

            if (btc_signal in ["BUY", "STRONG_BUY", "NEUTRAL"] and signal["signal"] == "STRONG_BUY"):
                stop_loss = current_price - (self.stop_loss_multiplier * current_atr)
                take_profit = current_price + (self.take_profit_multiplier * current_atr)
                print(f"🚀 LONG {self.symbol} | {current_time} | BTC: {btc_signal} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
                self.buy(sl=stop_loss, tp=take_profit)

            elif (btc_signal in ["SELL", "STRONG_SELL", "NEUTRAL"] and signal["signal"] == "STRONG_SELL"):
                stop_loss = current_price + (self.stop_loss_multiplier * current_atr)
                take_profit = current_price - (self.take_profit_multiplier * current_atr)
                print(f"🚨 SHORT {self.symbol} | {current_time} | BTC: {btc_signal} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
                self.sell(sl=stop_loss, tp=take_profit)




    def notify_trade(self, trade):
        """Log trade results when a trade is closed"""
        if trade.is_closed:
            trade_log.append({
                "Symbol": self.symbol,
                "Entry Time": trade.entry_time,
                "Exit Time": trade.exit_time,
                "Entry Price": trade.entry_price,
                "Exit Price": trade.exit_price,
                "Return [%]": trade.pl_pct * 100,
            })






import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy


results_list = []


for symbol in klines_df["Symbol"].unique():
    print(f"🔍 Testing {symbol}...")

    data = klines_df[klines_df["Symbol"] == symbol].copy()
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index("Date", inplace=True)

    if not isinstance(data.index, pd.DatetimeIndex):
        # print(f"⚠️ Index for {symbol} is not DatetimeIndex, attempting to convert...")
        data.index = pd.to_datetime(data.index)

    data.drop(columns=["Symbol"], inplace=True)

    symbol_signals = signals_df[(signals_df["symbol"] == symbol) & (signals_df["timeframe"] == f"{interval}")]
    unique_signals_count = symbol_signals["utc_time"].nunique()

    if symbol_signals.empty:
        print(f"⚠️ No signals for {symbol}, skipping...")
        continue

    print(f"📊 Found {len(symbol_signals)} signals ({unique_signals_count} unique) for {symbol}")

    SignalStrategy.signals_df = signals_df
    SignalStrategy.symbol = symbol

    bt = Backtest(data, SignalStrategy, cash=1_000_000, commission=0.0005)
    stats = bt.run()

    total_profit = sum(t["Return [%]"] for t in trade_log if t["Return [%]"] > 0)
    total_loss = sum(t["Return [%]"] for t in trade_log if t["Return [%]"] < 0)

    results = {
        "Symbol": symbol,
        "Total Trades": stats["# Trades"],
        "Win Rate": stats["Win Rate [%]"],
        "Return [%]": stats["Return [%]"],
        "Total Profit [%]": total_profit,
        "Total Loss [%]": total_loss,
        "Final Return [%]": total_profit + total_loss
    }
    results_list.append(results)

    print(f"✅ {symbol} — Return: {results['Return [%]']:.2f}%, Trades: {results['Total Trades']}, Win Rate: {results['Win Rate']:.2f}%, Profit: {results['Total Profit [%]']:.2f}%, Loss: {results['Total Loss [%]']:.2f}%\n")

results_df = pd.DataFrame(results_list)
results_df.to_csv(f"backtest_results_{interval}_BTC.csv", index=False)
trade_log_df = pd.DataFrame(trade_log)
trade_log_df.to_csv("trade_log.csv", index=False)

print("📌 Backtest completed! Results saved in backtest_results.csv")
print("📌 Trade log saved in trade_log.csv")

display(results_df)


🔍 Testing BNBUSDT...
📊 Found 86 signals (63 unique) for BNBUSDT


Backtest.run:   0%|          | 0/999 [00:00<?, ?it/s]

🚀 LONG BNBUSDT | 2025-03-05 21:00:00+00:00 | BTC: STRONG_BUY | Price: 595.91 | SL: 594.6354 | TP: 599.7339
🚀 LONG BNBUSDT | 2025-03-05 23:00:00+00:00 | BTC: STRONG_BUY | Price: 597.86 | SL: 596.7420 | TP: 601.2140
🚀 LONG BNBUSDT | 2025-03-06 02:00:00+00:00 | BTC: STRONG_BUY | Price: 602.09 | SL: 600.8628 | TP: 605.7717
🚀 LONG BNBUSDT | 2025-03-06 04:00:00+00:00 | BTC: BUY | Price: 603.5 | SL: 602.3386 | TP: 606.9842
🚀 LONG BNBUSDT | 2025-03-06 05:00:00+00:00 | BTC: STRONG_BUY | Price: 608.6 | SL: 607.4157 | TP: 612.1528
🚀 LONG BNBUSDT | 2025-03-06 06:00:00+00:00 | BTC: STRONG_BUY | Price: 608.04 | SL: 606.7610 | TP: 611.8769
🚀 LONG BNBUSDT | 2025-03-06 07:00:00+00:00 | BTC: BUY | Price: 606.0 | SL: 604.7326 | TP: 609.8023
🚨 SHORT BNBUSDT | 2025-03-06 17:00:00+00:00 | BTC: SELL | Price: 594.11 | SL: 595.5654 | TP: 589.7437
🚨 SHORT BNBUSDT | 2025-03-07 01:00:00+00:00 | BTC: STRONG_SELL | Price: 590.2 | SL: 592.2895 | TP: 583.9315
🚀 LONG BNBUSDT | 2025-03-07 06:00:00+00:00 | BTC: NEUTRAL 

/usr/local/lib/python3.11/dist-packages/backtesting/backtesting.py:955: UserWarning: time=445: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


🚨 SHORT BNBUSDT | 2025-03-08 22:00:00+00:00 | BTC: SELL | Price: 594.63 | SL: 595.4935 | TP: 592.0396
🚨 SHORT BNBUSDT | 2025-03-09 00:00:00+00:00 | BTC: SELL | Price: 592.13 | SL: 592.9172 | TP: 589.7685
🚨 SHORT BNBUSDT | 2025-03-09 01:00:00+00:00 | BTC: NEUTRAL | Price: 591.01 | SL: 591.7702 | TP: 588.7295
🚨 SHORT BNBUSDT | 2025-03-09 02:00:00+00:00 | BTC: SELL | Price: 589.03 | SL: 589.8333 | TP: 586.6202
🚨 SHORT BNBUSDT | 2025-03-09 06:00:00+00:00 | BTC: STRONG_SELL | Price: 585.06 | SL: 585.8426 | TP: 582.7123
🚨 SHORT BNBUSDT | 2025-03-09 10:00:00+00:00 | BTC: STRONG_SELL | Price: 581.94 | SL: 582.7286 | TP: 579.5741
🚨 SHORT BNBUSDT | 2025-03-09 11:00:00+00:00 | BTC: SELL | Price: 577.15 | SL: 578.0508 | TP: 574.4477
🚨 SHORT BNBUSDT | 2025-03-09 14:00:00+00:00 | BTC: STRONG_SELL | Price: 568.99 | SL: 570.0960 | TP: 565.6721
🚨 SHORT BNBUSDT | 2025-03-09 16:00:00+00:00 | BTC: SELL | Price: 567.31 | SL: 568.5985 | TP: 563.4446
🚨 SHORT BNBUSDT | 2025-03-09 17:00:00+00:00 | BTC: SELL | 

...

/usr/local/lib/python3.11/dist-packages/backtesting/backtesting.py:955: UserWarning: time=453: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


🚨 SHORT KDAUSDT | 2025-03-07 16:00:00+00:00 | BTC: SELL | Price: 0.4863 | SL: 0.4894 | TP: 0.4769
🚨 SHORT KDAUSDT | 2025-03-07 17:00:00+00:00 | BTC: SELL | Price: 0.488 | SL: 0.4913 | TP: 0.4780
🚀 LONG KDAUSDT | 2025-03-07 19:00:00+00:00 | BTC: NEUTRAL | Price: 0.5018 | SL: 0.4984 | TP: 0.5120
🚨 SHORT KDAUSDT | 2025-03-07 22:00:00+00:00 | BTC: STRONG_SELL | Price: 0.4828 | SL: 0.4862 | TP: 0.4725
🚨 SHORT KDAUSDT | 2025-03-08 01:00:00+00:00 | BTC: SELL | Price: 0.4826 | SL: 0.4857 | TP: 0.4732
🚨 SHORT KDAUSDT | 2025-03-08 03:00:00+00:00 | BTC: SELL | Price: 0.4798 | SL: 0.4826 | TP: 0.4714
🚨 SHORT KDAUSDT | 2025-03-08 05:00:00+00:00 | BTC: SELL | Price: 0.4811 | SL: 0.4836 | TP: 0.4736
🚨 SHORT KDAUSDT | 2025-03-08 06:00:00+00:00 | BTC: SELL | Price: 0.4804 | SL: 0.4827 | TP: 0.4734
🚨 SHORT KDAUSDT | 2025-03-08 07:00:00+00:00 | BTC: SELL | Price: 0.4787 | SL: 0.4809 | TP: 0.4720
🚨 SHORT KDAUSDT | 2025-03-08 08:00:00+00:00 | BTC: STRONG_SELL | Price: 0.4738 | SL: 0.4760 | TP: 0.4673
🚨 SHO

Backtest.run:   0%|          | 0/999 [00:00<?, ?it/s]

🚀 LONG ACTUSDT | 2025-03-05 17:00:00+00:00 | BTC: BUY | Price: 0.1959 | SL: 0.1951 | TP: 0.1982
🚀 LONG ACTUSDT | 2025-03-05 21:00:00+00:00 | BTC: STRONG_BUY | Price: 0.1967 | SL: 0.1960 | TP: 0.1987
🚀 LONG ACTUSDT | 2025-03-06 05:00:00+00:00 | BTC: STRONG_BUY | Price: 0.1987 | SL: 0.1981 | TP: 0.2006
🚀 LONG ACTUSDT | 2025-03-06 06:00:00+00:00 | BTC: STRONG_BUY | Price: 0.1957 | SL: 0.1950 | TP: 0.1979
🚨 SHORT ACTUSDT | 2025-03-06 09:00:00+00:00 | BTC: NEUTRAL | Price: 0.1941 | SL: 0.1947 | TP: 0.1922
🚨 SHORT ACTUSDT | 2025-03-06 13:00:00+00:00 | BTC: SELL | Price: 0.1937 | SL: 0.1943 | TP: 0.1919
🚨 SHORT ACTUSDT | 2025-03-06 17:00:00+00:00 | BTC: SELL | Price: 0.1944 | SL: 0.1950 | TP: 0.1925
🚨 SHORT ACTUSDT | 2025-03-06 22:00:00+00:00 | BTC: NEUTRAL | Price: 0.195 | SL: 0.1955 | TP: 0.1934
🚨 SHORT ACTUSDT | 2025-03-07 03:00:00+00:00 | BTC: STRONG_SELL | Price: 0.1943 | SL: 0.1949 | TP: 0.1925
🚨 SHORT ACTUSDT | 2025-03-07 06:00:00+00:00 | BTC: NEUTRAL | Price: 0.1929 | SL: 0.1934 | TP:

Backtest.run:   0%|          | 0/999 [00:00<?, ?it/s]

In [ ]:
# Фильтруем сигналы по BNB и BTC
bnb_signals = signals_df[signals_df["symbol"] == "CAKEUSDT"].copy()
btc_signals = signals_df[signals_df["symbol"] == "BTCUSDT"].copy()

# Приводим время к минутам
bnb_signals["utc_time"] = pd.to_datetime(bnb_signals["utc_time"]).dt.floor("min")
btc_signals["utc_time"] = pd.to_datetime(btc_signals["utc_time"]).dt.floor("min")

# Объединяем таблицы по utc_time
merged_signals = pd.merge(bnb_signals, btc_signals, on="utc_time", how="left", suffixes=('_bnb', '_btc'))

# Определяем, когда открылись сделки (учитываем NEUTRAL)
merged_signals["trade_opened_long"] = merged_signals.apply(
    lambda row: row["signal_btc"] in ["BUY", "STRONG_BUY", "NEUTRAL"] and row["signal_bnb"] == "STRONG_BUY",
    axis=1
)

merged_signals["trade_opened_short"] = merged_signals.apply(
    lambda row: row["signal_btc"] in ["SELL", "STRONG_SELL", "NEUTRAL"] and row["signal_bnb"] == "STRONG_SELL",
    axis=1
)

# Сохраняем в CSV для анализа
merged_signals.to_csv("signals_comparison.csv", index=False)

# Выведем первые 20 строк для проверки
merged_signals.head(20)


,#_bnb,id_bnb,symbol_bnb,timeframe_bnb,signal_bnb,entry_price_bnb,unix_timestamp_bnb,utc_time,local_time_bnb,#_btc,id_btc,symbol_btc,timeframe_btc,signal_btc,entry_price_btc,unix_timestamp_btc,local_time_btc,trade_opened_long,trade_opened_short
0,271,297,CAKEUSDT,1h,STRONG_SELL,2.155000,1.740575e+09,2025-02-26 13:00:00+00:00,2025-02-26 20:04:28.781618,295,321,BTCUSDT,1h,SELL,87742.700000,1.740575e+09,2025-02-26 20:05:08.624347,False,True
1,402,428,CAKEUSDT,1h,STRONG_SELL,2.133400,1.740579e+09,2025-02-26 14:00:00+00:00,2025-02-26 21:03:24.400815,338,364,BTCUSDT,1h,STRONG_SELL,87012.600000,1.740578e+09,2025-02-26 21:00:52.558900,False,True
2,139,876,CAKEUSDT,1h,STRONG_SELL,2.045100,1.740704e+09,2025-02-28 01:00:00+00:00,2025-02-28 08:00:34.000000,176,986,BTCUSDT,1h,STRONG_SELL,84020.969074,1.740706e+09,2025-02-28 08:23:44.000000,False,True
3,1009,1009,CAKEUSDT,1h,STRONG_SELL,1.959600,1.740928e+09,2025-03-02 15:00:00+00:00,2025-03-02 22:00:35.000000,1093,1093,BTCUSDT,1h,SELL,85081.330578,1.740928e+09,2025-03-02 22:02:02.000000,False,True
4,2115,2115,CAKEUSDT,1h,STRONG_SELL,1.969300,1.740964e+09,2025-03-03 01:00:00+00:00,2025-03-03 08:00:29.000000,2120,2120,BTCUSDT,1h,BUY,93309.200000,1.740964e+09,2025-03-03 08:00:37.000000,False,False
5,2167,2167,CAKEUSDT,1h,STRONG_SELL,1.923400,1.740974e+09,2025-03-03 04:00:00+00:00,2025-03-03 11:00:09.000000,2177,2177,BTCUSDT,1h,BUY,92667.903504,1.740974e+09,2025-03-03 11:00:24.000000,False,False
6,2243,2243,CAKEUSDT,1h,STRONG_SELL,1.855214,1.740985e+09,2025-03-03 07:00:00+00:00,2025-03-03 14:00:29.000000,2224,2224,BTCUSDT,1h,BUY,91653.100000,1.740985e+09,2025-03-03 14:00:07.000000,False,False
7,2491,2491,CAKEUSDT,1h,STRONG_SELL,1.850389,1.741011e+09,2025-03-03 14:00:00+00:00,2025-03-03 21:01:42.000000,2496,2496,BTCUSDT,1h,BUY,93349.400000,1.741011e+09,2025-03-03 21:02:00.000000,False,False
8,2502,2502,CAKEUSDT,1h,STRONG_SELL,1.793800,1.741014e+09,2025-03-03 15:00:00+00:00,2025-03-03 22:00:04.000000,2684,2684,BTCUSDT,1h,SELL,89830.948222,1.741014e+09,2025-03-03 22:03:15.000000,False,True
9,2874,2874,CAKEUSDT,1h,STRONG_SELL,1.808800,1.741025e+09,2025-03-03 18:00:00+00:00,2025-03-04 01:00:50.000000,2871,2871,BTCUSDT,1h,NEUTRAL,90367.500000,1.741025e+09,2025-03-04 01:00:47.000000,False,True


In [ ]:
long_trades = merged_signals["trade_opened_long"].sum()
short_trades = merged_signals["trade_opened_short"].sum()

print(f"📊 Итоговый подсчет сделок:")
print(f"🚀 Лонгов открыто: {long_trades}")
print(f"🚨 Шортов открыто: {short_trades}")

📊 Итоговый подсчет сделок:
🚀 Лонгов открыто: 0
🚨 Шортов открыто: 20


## Backtesting

In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy


results_list = []


for symbol in klines_df["Symbol"].unique():
    print(f"🔍 Testing {symbol}...")

    data = klines_df[klines_df["Symbol"] == symbol].copy()
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index("Date", inplace=True)

    if not isinstance(data.index, pd.DatetimeIndex):
        print(f"⚠️ Index for {symbol} is not DatetimeIndex, attempting to convert...")
        data.index = pd.to_datetime(data.index)

    data.drop(columns=["Symbol"], inplace=True)

    symbol_signals = signals_df[(signals_df["symbol"] == symbol) & (signals_df["timeframe"] == f"{interval}")]
    unique_signals_count = symbol_signals["utc_time"].nunique()

    if symbol_signals.empty:
        print(f"⚠️ No signals for {symbol}, skipping...")
        continue

    print(f"📊 Found {len(symbol_signals)} signals ({unique_signals_count} unique) for {symbol}")

    SignalStrategy.signals_df = signals_df
    SignalStrategy.symbol = symbol

    bt = Backtest(data, SignalStrategy, cash=1_000_000, commission=0.0005)
    stats = bt.run()

    total_profit = sum(t["Return [%]"] for t in trade_log if t["Return [%]"] > 0)
    total_loss = sum(t["Return [%]"] for t in trade_log if t["Return [%]"] < 0)

    results = {
        "Symbol": symbol,
        "Total Trades": stats["# Trades"],
        "Win Rate": stats["Win Rate [%]"],
        "Return [%]": stats["Return [%]"],
        "Total Profit [%]": total_profit,
        "Total Loss [%]": total_loss,
        "Final Return [%]": total_profit + total_loss
    }
    results_list.append(results)

    print(f"✅ {symbol} — Return: {results['Return [%]']:.2f}%, Trades: {results['Total Trades']}, Win Rate: {results['Win Rate']:.2f}%, Profit: {results['Total Profit [%]']:.2f}%, Loss: {results['Total Loss [%]']:.2f}%\n")

results_df = pd.DataFrame(results_list)
results_df.to_csv(f"backtest_results_{interval}.csv", index=False)
trade_log_df = pd.DataFrame(trade_log)
trade_log_df.to_csv("trade_log.csv", index=False)

print("📌 Backtest completed! Results saved in backtest_results.csv")
print("📌 Trade log saved in trade_log.csv")

display(results_df)


🔍 Testing BNBUSDT...
📊 Found 48 signals (48 unique) for BNBUSDT


TypeError: 'NoneType' object is not subscriptable

### Hypeopt_by_month

In [ ]:
symbol_1 = 'BTCUSDT'
interval_1 = '1h'
start_date_1 = '2023-12'
end_date_1 = '2025-02'
max_evals_1 = 50


# Параметры для оптимизации
params_to_optimize_1 = {
    'atr_period': {'type': 'quniform', 'range': (7, 50, 1)},
    'multiplier': {'type': 'uniform', 'range': (1.0, 3.5)},
    'ema_period': {'type': 'quniform', 'range': (7, 50, 1)}

}

fixed_params_1 = {}
# # Фиксированные параметры
# fixed_params_1 = {
#     'min_space': 5,
#     # 'window_length': 50,
#     # 'sloping_atr_length': 20,
#     # 'take_profit_multiplier': 1.5,
#     # 'stop_loss_multiplier': 0.5
# }
fixed_params_1 = {}

In [ ]:
# Настройка и вызов оптимизации
# symbol = 'SUIUSDT'
# interval = '1h'
# start_date = '2024-11'
# end_date = '2024-11'

symbol = symbol_1
interval = interval_1
start_date = start_date_1
end_date = end_date_1
max_evals = max_evals_1
params_to_optimize=params_to_optimize_1
fixed_params=fixed_params_1
strategy = SuperTrendStrategy

# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(symbol, interval)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")

## Final data

In [ ]:
import pandas as pd

klines_df = pd.read_csv("klines_data_1h.csv")
signals_df = pd.read_csv("tradingview_signals_server.csv")
interval = '1h'

df = signals_df[(signals_df["timeframe"] == f"{interval}")]
df.sort_values(by=["symbol", 'utc_time'], inplace=True)
df.to_csv(f"signals_{interval}.csv", index=False)
df

<ipython-input-6-64dd86021997>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=["symbol", 'utc_time'], inplace=True)


,#,id,symbol,timeframe,signal,entry_price,unix_timestamp,utc_time,local_time
234,235,235,1000CATUSDT,1h,STRONG_SELL,0.009621,1740855634,2025-03-01 19:00:34.000000,2025-03-02 02:00:34.000000
296,297,297,1000CATUSDT,1h,STRONG_SELL,0.009602,1740859218,2025-03-01 20:00:18.000000,2025-03-02 03:00:18.000000
1490,1491,1491,1000CATUSDT,1h,STRONG_BUY,0.010320,1740935011,2025-03-02 17:03:31.000000,2025-03-03 00:03:31.000000
1638,1639,1639,1000CATUSDT,1h,STRONG_BUY,0.010530,1740938482,2025-03-02 18:01:22.000000,2025-03-03 01:01:22.000000
1767,1768,1768,1000CATUSDT,1h,STRONG_BUY,0.010600,1740942037,2025-03-02 19:00:37.000000,2025-03-03 02:00:37.000000
...,...,...,...,...,...,...,...,...,...
27415,27416,27416,ZRXUSDT,1h,STRONG_SELL,0.253400,1741536006,2025-03-09 16:00:06.000000,2025-03-09 23:00:06.000000
28300,28301,28301,ZRXUSDT,1h,STRONG_SELL,0.247200,1741539613,2025-03-09 17:00:13.000000,2025-03-10 00:00:13.000000
29211,29212,29212,ZRXUSDT,1h,STRONG_SELL,0.244000,1741543229,2025-03-09 18:00:29.000000,2025-03-10 01:00:29.000000
30151,30152,30152,ZRXUSDT,1h,STRONG_SELL,0.241900,1741557632,2025-03-09 22:00:32.000000,2025-03-10 05:00:32.000000
